In [220]:
!pip install -q lightning flwr wandb hydra-core

In [221]:
from torch.utils.data import Dataset
from pathlib import Path
import nibabel as nib
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from typing import Dict, List, Optional, Tuple, Union

In [222]:
from kaggle_secrets import UserSecretsClient
secret_label = "wandb_api_key"
WANDB_APIKEY = UserSecretsClient().get_secret(secret_label)

ROOT_PATH = '/kaggle/input/mini-brain3d-dataset/not_skull_stripped'

# Dataset

In [223]:
class MRIDataset(Dataset) :

    def __init__(self, root_dir: str, label_path: str = None, transform = None, label_df: pd.DataFrame = None, is_3d: bool = False):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.is_3d = is_3d
        if label_df is None:
          self.labels_df = pd.read_csv(label_path)
          
        else :
          self.labels_df = label_df

        self.labels_df['subject_id'] = self.labels_df['subject_id'].astype(str)
        self.labels_df = self.labels_df[self.labels_df['subject_dx'] == 'control']

        all_nii_files = list(self.root_dir.rglob("*.nii"))
        fail_paths = ["sub-BrainAge005600/anat/sub-BrainAge005600_T1w.nii/sub-BrainAge005600_T1w.nii"]
        self.file_paths = [fp for fp in all_nii_files if fp.is_file() and fp.name not in fail_paths ]

        valid_subjects = set(self.labels_df['subject_id'].values)

        self.file_paths = [fp for fp in self.file_paths if any(vs in str(fp) for vs in valid_subjects)]
        self.file_paths.sort()



    def __len__(self):
        return len(self.file_paths)


    def preprocessing_datapoint(self, img_data):

        mid_x = img_data.shape[0] // 2
        mid_y = img_data.shape[1] // 2
        mid_z = img_data.shape[2] // 2

        axial_slice = img_data[:, :, mid_z]
        coronal_slice = img_data[:, mid_y, :]
        sagittal_slice = img_data[mid_x, :, :]


        combined_data = np.stack([axial_slice, coronal_slice, sagittal_slice], axis=0)
        combined_data = torch.from_numpy(combined_data).float()

        if self.transform : combined_data = self.transform(combined_data)

        return combined_data




    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        file_path_str = str(img_path)

        subject_id = None
        valid_subjects_set = set(self.labels_df['subject_id'].values)


        for sid in valid_subjects_set:
            if sid in file_path_str:
                subject_id = sid
                break

        if subject_id is None:
            raise ValueError(f"Không tìm thấy subject_id cho file: {img_path}")

        metadata = self.labels_df.loc[self.labels_df['subject_id'] == subject_id].iloc[0].to_dict()

        img_data = nib.load(img_path).get_fdata()

        img_data = torch.from_numpy(img_data).unsqueeze(0).float()

        label = 0
        if metadata['subject_sex'] == 'm' : label = 1

        if not self.is_3d:
            img_data = self.preprocessing_datapoint(img_data)

        return img_data,  label



def visualize_sample(dataset, idx):
    mri_data, label = dataset[idx]
    title = f"Label: {label}\n"
    plt.close('all')
    fig = plt.figure(figsize = (18, 6))

    if isinstance(mri_data, torch.Tensor):
        data = mri_data.squeeze().numpy()
    else:
        data = mri_data


    ax1 = fig.add_subplot(1, 3, 1)
    plt.imshow(data[0, :, :].T, cmap='gray', origin='lower')

    ax2 = fig.add_subplot(1, 3, 2)
    ax2.imshow(data[1, :, :].T, cmap='gray', origin='lower')

    ax3 = fig.add_subplot(1, 3, 3)
    ax3.imshow(data[2, :, :].T, cmap='gray', origin='lower')

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


## Data splitting

In [224]:
from torch.utils.data import random_split 

def preprocessing_labels(df: pd.DataFrame, root_dir: str = ROOT_PATH):
    subject_list = []
    for root, dirs, files in os.walk(root_dir):
      for dir_name in dirs:
        if dir_name.startswith("sub-BrainAge"):
            subject_list.append(dir_name)

    return df[df['subject_id'].isin(subject_list)]


def prepare_data(data: pd.DataFrame):
    df = data.copy()
    df['age_group'] = pd.qcut(df['subject_age'], q = min(5, len(df)), labels = False)
    df['key'] = df.apply(lambda row : f"{row['age_group']}_{row['subject_sex']}", axis = 1)
    return df


def sampling_data(data, size, random_state ):
  samples = data.groupby('key', group_keys = False)
  samples = samples.apply(lambda x: x.sample(
      n = min(int(size / len(data['key'].unique())), len(x)),
      replace = len(x) < int(size / len(data['key'].unique())),
      random_state =  random_state
  ), include_groups=False)


  if len(samples) < size:
    additional_samples = data.drop(samples.index).sample(
        n = min(size - len(samples), len(data) - len(samples)),
        replace = True,
        random_state = random_state
    )

    samples = pd.concat([samples, additional_samples])
  return samples


def create_train_test(sample_labels: list, val_ratio: float = 0.2, root_dir: str = ROOT_PATH, is_3d: bool = False):
    
  client_datasets = []
  for label_df in sample_labels:
    dataset = MRIDataset(root_dir=root_dir, label_df = label_df, is_3d = is_3d)
    
    train_dataset, val_dataset = random_split(dataset, [1 - val_ratio, val_ratio])
    client_datasets.append((train_dataset, val_dataset))
  return client_datasets
    

def distributed_data_to_clients(data: pd.DataFrame, num_clients: int, overlap_ratio: float):

  df = prepare_data(data)

  n_samples = len(df)
  samples_per_client = int(n_samples / (num_clients * (1 - overlap_ratio) + overlap_ratio))

  client_datasets = []
  selected_samples = {}

  # Tạo các client datasets với sự phân bố cân bằng
  for client_idx in range(num_clients):

      if client_idx == 0:
          client_data = df.sample(n=samples_per_client, random_state=42+client_idx)
      else:
          # overlap size
          overlap_size = int(samples_per_client * overlap_ratio)
          non_overlap_size = samples_per_client - overlap_size

          # building overlap
          all_previous_samples = pd.DataFrame()
          for prev_client_idx in range(client_idx):
              all_previous_samples = pd.concat([all_previous_samples, selected_samples[prev_client_idx]])

          # sampling
          if len(all_previous_samples) > 0:
              overlap_samples = sampling_data(all_previous_samples, overlap_size, client_idx * 100 + 42)
          else:
              overlap_samples = pd.DataFrame(columns=df.columns)

          # Lấy mẫu mới (không overlap)
          remaining_indices = df.index.difference(all_previous_samples.index)
          if len(remaining_indices) > 0:
              remaining_df = df.loc[remaining_indices]
              non_overlap_samples = sampling_data(remaining_df, non_overlap_size, client_idx * 100 + 42)
          else:

              non_overlap_samples = df.sample(n=non_overlap_size, replace=True, random_state=42+client_idx*300)


          client_data = pd.concat([overlap_samples, non_overlap_samples])


      selected_samples[client_idx] = client_data
      client_datasets.append(client_data.drop(['age_group', 'key'], axis=1))

  return client_datasets

In [225]:
def iid_client_split(dataset, num_client = 3,  val_ratio = 0.2):

    client_datasets = []
    sample_per_client = len(dataset) // num_client


    for i in range(num_client):
        start_idx = i * sample_per_client
        end_idx = (i + 1) * sample_per_client if i < num_client - 1 else len(dataset)
        indecies = list(range(start_idx, end_idx))

        client_dataset = torch.utils.data.Subset(dataset, indecies)
        train_dataset, val_dataset = random_split(client_dataset, [1 - val_ratio, val_ratio])

        client_datasets.append((train_dataset, val_dataset))
    return client_datasets


def same_distribution_client_split(dataset, num_client, val_ratio = 0.2, overlap_ratio = 0.2, root_dir = ROOT_PATH, is_3d = False):
    """
    Split the dataset into clients with the same distribution of labels.
    """
    labels_df = dataset.labels_df
    labels_df = preprocessing_labels(labels_df, root_dir = root_dir)    
    labels_df = prepare_data(labels_df)

    client_datasets = distributed_data_to_clients(labels_df, num_clients=num_client, overlap_ratio=overlap_ratio)

    client_datasets = create_train_test(client_datasets, val_ratio=val_ratio, root_dir=root_dir, is_3d = is_3d)

    return client_datasets


#  Model

In [226]:
import torch.nn as nn 
import torch.nn.functional as F
from collections import OrderedDict
from torchsummary import summary

class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super().__init__()
        self.add_module('norm1', nn.BatchNorm3d(num_input_features))
        self.add_module('relu1', nn.ReLU(inplace=True))
        self.add_module(
            'conv1',
            nn.Conv3d(num_input_features,
                      bn_size * growth_rate,
                      kernel_size=1,
                      stride=1,
                      bias=False))
        self.add_module('norm2', nn.BatchNorm3d(bn_size * growth_rate))
        self.add_module('relu2', nn.ReLU(inplace=True))
        self.add_module(
            'conv2',
            nn.Conv3d(bn_size * growth_rate,
                      growth_rate,
                      kernel_size=3,
                      stride=1,
                      padding=1,
                      bias=False))
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super().forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features,
                                     p=self.drop_rate,
                                     training=self.training)
        return torch.cat([x, new_features], 1)

class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super().__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate,
                                growth_rate, bn_size, drop_rate)
            self.add_module('denselayer{}'.format(i + 1), layer)

class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super().__init__()
        self.add_module('norm', nn.BatchNorm3d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module(
            'conv',
            nn.Conv3d(num_input_features,
                      num_output_features,
                      kernel_size=1,
                      stride=1,
                      bias=False))
        self.add_module('pool', nn.AvgPool3d(kernel_size=2, stride=2))

class DenseNet(nn.Module):
    def __init__(self,
                 n_input_channels=1,
                 conv1_t_size=7,
                 conv1_t_stride=1,
                 no_max_pool=False,
                 growth_rate=16,
                 block_config=(4, 8, 16, 12),
                 num_init_features=32,
                 bn_size=4,
                 drop_rate=0,
                 num_classes=2):
        super().__init__()

        # First convolution
        self.features = [('conv1',
                          nn.Conv3d(n_input_channels,
                                    num_init_features,
                                    kernel_size=(conv1_t_size, 7, 7),
                                    stride=(conv1_t_stride, 2, 2),
                                    padding=(conv1_t_size // 2, 3, 3),
                                    bias=False)),
                         ('norm1', nn.BatchNorm3d(num_init_features)),
                         ('relu1', nn.ReLU(inplace=True))]
        if not no_max_pool:
            self.features.append(
                ('pool1', nn.MaxPool3d(kernel_size=3, stride=2, padding=1)))
        self.features = nn.Sequential(OrderedDict(self.features))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers,
                                num_input_features=num_features,
                                bn_size=bn_size,
                                growth_rate=growth_rate,
                                drop_rate=drop_rate)
            self.features.add_module('denseblock{}'.format(i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition{}'.format(i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm3d(num_features))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

        # Khởi tạo trọng số
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool3d(out, output_size=(1, 1, 1)).view(features.size(0), -1)
        logits = self.classifier(out)
        return out, logits
    

## Relaxed Balanced Softmax & feature augmentation

In [227]:
class RelaxedBSM(nn.Module):
    def __init__(self, dataloader, num_classes, eps=0.01, device=None):
        super(RelaxedBSM, self).__init__()
        self.num_classes = num_classes
        self.dataloader = dataloader
        self.eps = eps
        
        self.device = device if device is not None else torch.device('cuda' if torch.cuda.is_available() else "cpu")
        self._cached_prior = None
        self.to(self.device)

    def prior_y(self, recalculate=False):
        if self._cached_prior is not None and not recalculate:
            return self._cached_prior
            
        py = torch.zeros(self.num_classes, device=self.device)
        total_samples = 0
        with torch.no_grad():
            for _, labels in self.dataloader:
                labels = labels.to(self.device)

                if not labels.dtype.is_floating_point:
                    bin_counts = torch.bincount(labels, minlength=self.num_classes).float()
                else:
                    bin_counts = labels.sum(dim=0) if labels.dim() > 1 \
                    else torch.zeros(self.num_classes, device=self.device).scatter_add_(0, labels.long(), torch.ones_like(labels, device=self.device))

                py += bin_counts
                total_samples += labels.size(0)
        self._cached_prior = py / total_samples if total_samples > 0 else torch.ones(self.num_classes, device=self.device) / self.num_classes
        return self._cached_prior

    def smooth_distribution(self, py):
        """Apply smoothing to a probability distribution"""
        py_smooth = (1 - self.eps) * py + self.eps / self.num_classes
        return py_smooth / py_smooth.sum()

    def prior_y_batch(self, labels):
        labels = labels.to(self.device)
        py = torch.bincount(labels, minlength=self.num_classes).float()
        return py / labels.size(0)

    def bsm1(self, logit):
        py = self.prior_y()
        py_smooth = self.smooth_distribution(py)

        logit = logit - logit.max(dim=1, keepdim=True)[0]
        exp_logits = torch.exp(logit)
        
        pc_exp = exp_logits * py_smooth.unsqueeze(0)
        denominator = pc_exp.sum(dim=1, keepdim=True) + 1e-8
        
        return pc_exp / denominator

    def bsm2(self, logit, py):
        """Balanced Softmax with provided class distribution"""
        py = py.to(self.device)
        py_smooth = self.smooth_distribution(py)
        
        logit = logit - logit.max(dim=1, keepdim=True)[0]
        exp_logits = torch.exp(logit)
        
        pc_exp = exp_logits * py_smooth.unsqueeze(0)
        denominator = pc_exp.sum(dim=1, keepdim=True) + 1e-8
        
        return pc_exp / denominato
    
    def forward(self, logit, py=None):
        """Forward method for nn.Module compatibility"""
        if py is None:
            return self.bsm1(logit)
        else:
            return self.bsm2(logit, py)

In [228]:
def get_prototypes(model, dataloader, num_classes, device=None):
    model.eval()
    prototypes = [[] for _ in range(num_classes)]
    with torch.no_grad():
        for im, labels in dataloader:
            im = im.to(device, non_blocking = True)
            labels = labels.to(device, non_blocking = True)
            model = model.to(device)
            features, _ = model(im)
            for i, label in enumerate(labels):
                prototypes[label.item()].append(features[i])
                
    class_prototypes = torch.zeros((num_classes, features.shape[1]), device = device)
    for c in range(num_classes):
        if prototypes[c]:
            # average over all feature vectors from the same class
            class_prototypes[c] = torch.stack(prototypes[c]).mean(dim = 0)
        else:
            class_prototypes[c] = torch.zeros(features.shape[1], device = device) # pseudo-prototypes for missing class in dataloader
            
    return class_prototypes # [num_classes, feature_vectors]

def feature_augmentation(features, labels, prototypes, num_classes, lam=1.0, device=None):
    """Perform feature augmentation by transferring intra-class variance to missing classes."""
    aug_features = []
    aug_labels = []
    for i in range(len(labels)):
        src_class = labels[i].item()
        # Cycle through classes for augmentation
        tgt_class = (src_class + 1) % num_classes
        if torch.all(prototypes[tgt_class] == 0):
            continue
        # Compute augmented feature: \tilde{h}_{j,k} = p_j + \lambda (h_{i,k} - p_i)
        aug_feature = prototypes[tgt_class].to(device) + lam * (features[i] - prototypes[src_class].to(device))
        aug_features.append(aug_feature)
        aug_labels.append(tgt_class)
    
    if aug_features:
        aug_features = torch.stack(aug_features)
        aug_labels = torch.tensor(aug_labels, dtype=torch.long, device=features.device)
        return aug_features, aug_labels
    else:
        return None, None

## Lightning module

In [229]:
import lightning as pl
from torchmetrics import Accuracy, F1Score, Precision, Recall, MeanMetric
import torch.optim as optim 


class DenseNetModule(pl.LightningModule):
    def __init__(self, net, rbsm=None, global_prototypes=None, learning_rate=1e-3, weight_decay = 1e-2, batch_size = 32, lam=1, mu=0.1):
        super().__init__()
        self.model = net
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.batch_size = batch_size
        if global_prototypes:
            self.global_prototypes = global_prototypes
        else:
            self.global_prototypes = [torch.zeros(364) for _ in range(2)]
        self.lam = lam
        self.mu = mu
        
        # how confidence model is in it prediction
        # tức model có thể rất tự tin trong quyết định nhưng thực tế lại sai
        # BCE = y*log(y_pred) + (1 - y)*log(1 - y_pred)
        self.criterion = nn.CrossEntropyLoss()
        self.nll_loss = nn.NLLLoss()
        self.rbsm = rbsm
        
        self.train_accuracy = Accuracy(task="multiclass", num_classes=2)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=2)
        self.test_accuracy = Accuracy(task="multiclass", num_classes=2)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        features, logits = self(x)
        
        balanced_probs = self.rbsm(logits)
        loglikelihood = torch.log(balanced_probs + 1e-8)
        loss0 = self.nll_loss(loglikelihood, y)
        loss1 = 0.0
        aug_features, aug_labels = feature_augmentation(features, y, self.global_prototypes, 2, self.lam, self.device)
        if aug_features is not None: 
            aug_logits = self.model.classifier(aug_features)
            aug_probs = self.rbsm(aug_logits)
            aug_loglikelihood = torch.log(aug_probs + 1e-8)
            loss1 = self.nll_loss(aug_loglikelihood, aug_labels)

        loss = loss0 + self.mu * loss1
        acc = self.train_accuracy(torch.argmax(logits, dim=1), y)

        self.log('train/loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('train/acc', acc, on_step=False, on_epoch=True, prog_bar=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        _, logits = self(x)
        
        loss = self.criterion(logits, y)
        acc = self.val_accuracy(torch.argmax(logits, dim=1), y)
        
        self.log('val/loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val/acc', acc, on_step=False, on_epoch=True, prog_bar=True)

        return loss


    def test_step(self, batch, batch_idx):
        x, y = batch
        _, logits = self(x)
        
        loss = self.criterion(logits, y)
        acc = self.test_accuracy(torch.argmax(logits, dim=1), y)

        self.log('test/loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('test/acc', acc, on_step=False, on_epoch=True, prog_bar=True)
        return loss
            
    def configure_optimizers(self):
        optimizer =  torch.optim.SGD(self.parameters(), lr=self.learning_rate, weight_decay = self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)

        return {
           "optimizer": optimizer,
           "lr_scheduler": {
               "scheduler": scheduler,
               "monitor": "val_loss",
           },
        }


# Client

In [230]:
from collections import OrderedDict
from flwr.client import NumPyClient
from flwr.common import  Context
from torch.utils.data import DataLoader
import logging 
import warnings 
from lightning.pytorch.loggers.wandb import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
import pickle

In [231]:
class FlowerLightningClient(NumPyClient):

    def __init__(self, model: pl.LightningModule, train_dataloader, val_dataloader, epochs, batch_size, device, client_id): 
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.epochs = epochs
        self.device = device 
        self.client_id = client_id
        self.model = model
        self.batch_size = batch_size


    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        if not parameters:
            return
    
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict((k, torch.tensor(v)) for k, v in params_dict)
    
        if state_dict:
            self.model.load_state_dict(state_dict, strict=False)

        self.model = self.model.to(self.device)

        
    def fit(self, parameters, config):

        self.set_parameters(parameters)
        
        checkpoint_callback = ModelCheckpoint(
            dirpath=f"./checkpoints/client_{self.client_id}",
            filename=f"round_{config.get('round_num', 0)}" + "-{epoch:02d}",
            save_top_k=1,
            monitor="val/loss",
            mode="min"
        )

        trainer = pl.Trainer(
            max_epochs=self.epochs,
            accelerator="auto",
            devices=1,
            callbacks=[checkpoint_callback],
            enable_progress_bar=False, 
            log_every_n_steps=1
        )
        
        global_prototypes = pickle.loads(config['global_prototypes'])
        self.model.global_prototypes = global_prototypes
        rbsm = RelaxedBSM(self.train_dataloader, num_classes=2, device=self.device)
        self.model.rbsm = rbsm.to(self.device)
        self.model.model = self.model.model.to(self.device)
        trainer.fit(self.model.to(self.device), train_dataloaders=self.train_dataloader, val_dataloaders=self.val_dataloader)
        local_prototypes = get_prototypes(self.model, self.train_dataloader, 2, self.device)
        
        callback_metrics = trainer.callback_metrics

        train_loss = callback_metrics.get("train/loss", 0)
        train_accuracy = callback_metrics.get("train/acc", 0)
        val_loss = callback_metrics.get("val/loss", 0)
        val_accuracy = callback_metrics.get("val/acc", 0)
    
        # Calculate label counts
        from collections import Counter
        label_counts = Counter()
        for _, label in self.train_dataloader.dataset:
            if isinstance(label, torch.Tensor):
                label = label.item()
            label_counts[label] += 1
            
        metrics = {
            "train_loss": train_loss.item() if isinstance(train_loss, torch.Tensor) else float(train_loss),
            "train_accuracy": train_accuracy.item() if isinstance(train_accuracy, torch.Tensor) else float(train_accuracy),
            "val_loss": val_loss.item() if isinstance(val_loss, torch.Tensor) else float(val_loss),
            "val_accuracy": val_accuracy.item() if isinstance(val_accuracy, torch.Tensor) else float(val_accuracy),
            "local_prototypes": pickle.dumps(local_prototypes),
            "label_counts": pickle.dumps(label_counts)
        }

        return self.get_parameters(config={}), len(self.train_dataloader.dataset), metrics


    def evaluate(self, parameters, config):

        self.set_parameters(parameters)

        trainer = pl.Trainer(
            accelerator="auto",
            devices=1 ,
            enable_progress_bar=False
        )

        results = trainer.test(self.model, dataloaders=self.val_dataloader)
        
        callback_metrics = trainer.callback_metrics

        test_loss = callback_metrics.get("test/loss")
        test_accuracy = callback_metrics.get("test/acc")
        
        # Additional metrics
        metrics = {
            "test_loss": test_loss.item() if isinstance(test_loss, torch.Tensor) else float(test_loss),
            "test_accuracy": test_accuracy.item() if isinstance(test_accuracy, torch.Tensor) else float(test_accuracy),
        }
        return float(test_loss), len(self.val_dataloader.dataset), metrics


In [232]:
def create_lightning_client_fn(device, epochs, client_datasets, batch_size, num_workers, pl_model):

    def client_fn(context: Context):
        
        client_id = context.node_config['partition-id']
        train_dataset, val_dataset = client_datasets[client_id]

        train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = num_workers)
        val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = num_workers)

        return FlowerLightningClient(pl_model, train_dataloader, val_dataloader, epochs, batch_size, device, client_id).to_client()

    return client_fn

In [233]:
import random
import flwr as fl
from flwr.common import (
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    EvaluateIns,
)
from flwr.server.client_proxy import ClientProxy
from flwr.server.client_manager import ClientManager
from flwr.server.strategy import FedAvg
from flwr.common.parameter import parameters_to_ndarrays

In [234]:
from flwr.common import ndarrays_to_parameters

def get_parameters(model) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in model.state_dict().items()]
    
class DropoutFedAvg(FedAvg):
    """FedAvg strategy with client dropout simulation and metrics tracking."""

    def __init__( self, net, dropout_rate_training: float = 0.3, dropout_rate_eval: float = 0.3, fixed_clients: Optional[List[int]] = None, dropout_pattern_train: str = "random", dropout_pattern_eval: str = "random", **kwargs):
    
        if "fit_metrics_aggregation_fn" not in kwargs:
            kwargs["fit_metrics_aggregation_fn"] = self.weighted_average
        if "evaluate_metrics_aggregation_fn" not in kwargs:
            kwargs["evaluate_metrics_aggregation_fn"] = self.weighted_average

        super().__init__(**kwargs)
        self.dropout_rate_training = dropout_rate_training
        self.dropout_rate_eval = dropout_rate_eval
        self.fixed_clients = fixed_clients or []
        self.dropout_pattern_train = dropout_pattern_train
        self.dropout_pattern_eval = dropout_pattern_eval
        self.current_round = 0
        self.dropped_clients_history_training: Dict[int, List[int]] = {}
        self.dropped_clients_history_evaluation: Dict[int, List[int]] = {}

        # For tracking metrics
        self.fit_metrics_history: List[Dict[str, float]] = []
        self.eval_metrics_history: List[Dict[str, float]] = []

        self.net = net
        self.global_prototypes = [torch.zeros(364) for _ in range(2)] # create pseudo-global prototypes
    
    def weighted_average(self, metrics: List[Tuple[int, Dict]]) -> Dict:
        """Aggregate metrics using weighted average based on number of samples."""
        if not metrics:
            return {}

        total_examples = sum([num_examples for num_examples, _ in metrics])
        weighted_metrics = {}

        for metric_key in ["train_loss", "train_accuracy", "val_loss", "val_accuracy", "test_accuracy", "test_loss"]:
            values = [
                metric_dict[metric_key] * num_examples
                for num_examples, metric_dict in metrics
                if metric_key in metric_dict and isinstance(metric_dict[metric_key], (int, float))
            ]
            if values:
                weighted_sum = sum(values)
                weighted_metrics[metric_key] = weighted_sum / total_examples if total_examples > 0 else 0

        return weighted_metrics


    def configure_fit( self, server_round: int, parameters: Parameters, client_manager: ClientManager) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training with client dropout."""
        self.current_round = server_round

    
        client_fit_instructions = super().configure_fit(
            server_round, parameters, client_manager, 
        )

        if not client_fit_instructions:
            return []


        available_clients = self._apply_dropout(client_fit_instructions, dropout_rate=self.dropout_rate_training, dropout_pattern=self.dropout_pattern_train)

        # Save dropout history for this round
        client_ids = [int(client.cid) for client, _ in client_fit_instructions]
        available_client_ids = [int(client.cid) for client, _ in available_clients]
        dropped_clients = [cid for cid in client_ids if cid not in available_client_ids]
        self.dropped_clients_history_training[server_round] = dropped_clients

        print(f"Round {server_round}: {len(dropped_clients)} clients dropped out of {len(client_ids)} during training")
        print(f"Dropped client IDs: {dropped_clients}")

        # send global prototypes to clients
        updated_clients = []
        for client, fit_ins in available_clients:
            custom_config = dict(fit_ins.config)
            custom_config['global_prototypes'] = pickle.dumps(self.global_prototypes)
            new_fit_ins = FitIns(parameters=fit_ins.parameters, config=custom_config)
            updated_clients.append((client, new_fit_ins))
            
        return updated_clients
    
    def configure_evaluate( self, server_round: int, parameters: Parameters, client_manager: ClientManager) -> List[Tuple[ClientProxy, EvaluateIns]]:
        self.current_round = server_round

        client_evaluate_instructions = super().configure_evaluate(
            server_round, parameters, client_manager
        )

        if not client_evaluate_instructions: return []

        available_clients = self._apply_dropout(client_evaluate_instructions, dropout_rate=self.dropout_rate_eval, dropout_pattern=self.dropout_pattern_eval)

        client_ids = [int(client.cid) for client, _ in client_evaluate_instructions]
        available_client_ids = [int(client.cid) for client, _ in available_clients]
        dropped_clients = [cid for cid in client_ids if cid not in available_client_ids]

        self.dropped_clients_history_evaluation[server_round] = dropped_clients
        

        print(f"Round {server_round}: {len(dropped_clients)} clients dropped out of {len(client_ids)} during evaluation")
        print(f"Dropped client IDs: {dropped_clients}")

        return available_clients


    def _apply_dropout(self, client_instructions: List[Tuple[ClientProxy, Union[FitIns, EvaluateIns ]]], dropout_pattern: str, dropout_rate: 0.3) -> List[Tuple[ClientProxy, FitIns]]:
        """Apply dropout to clients based on the specified pattern."""
        if len(client_instructions) == 0:
            return []

        # Get all client IDs
        all_clients = [(client, ins) for client, ins in client_instructions]
        all_client_ids = [int(client.cid) for client, _ in all_clients]

        # Determine which clients will drop out
        dropout_mask = [False] * len(all_clients)

        if dropout_pattern == "random":
           
            for i, cid in enumerate(all_client_ids):
                
                if cid in self.fixed_clients:
                    continue
            
                if random.random() < dropout_rate:
                    dropout_mask[i] = True

        elif dropout_pattern == "alternate":
         
            if self.current_round % 2 == 1:  
                for i, cid in enumerate(all_client_ids):
                    if cid not in self.fixed_clients:
                        dropout_mask[i] = True

        elif dropout_pattern == "fixed":
      
            n_dropout = int(len(all_clients) * dropout_rate)
            for i in range(n_dropout):
                if all_client_ids[i] not in self.fixed_clients:
                    dropout_mask[i] = True

        
        available_clients = [
            (client, ins) for i, (client, ins) in enumerate(all_clients)
            if not dropout_mask[i]
        ]

        return available_clients

    def l2dist(self, p1, p2):
        return np.linalg.norm(p1 - p2)
    
    def weighted_average_oracle(self, points, weights):
        tot_weights = np.sum(weights)
        weighted_updates = np.zeros_like(points[0])
        for w, p in zip(weights, points):
            weighted_updates += (w / tot_weights) * p
        return weighted_updates
    
    def geometric_median_objective(self, median, points, alphas):
        return sum([alpha * self.l2dist(median, p) for alpha, p in zip(alphas, points)])
    
    def geometric_median_update(self, points, alphas, maxiter=4, eps=1e-5, ftol=1e-6, verbose=False):
        alphas = np.asarray(alphas, dtype=points[0].dtype) / sum(alphas)
        median = self.weighted_average_oracle(points, alphas)
        for _ in range(maxiter):
            weights = np.asarray([
                alpha / max(eps, self.l2dist(median, p)) for alpha, p in zip(alphas, points)
            ], dtype=alphas.dtype)
            weights /= weights.sum()
            new_median = self.weighted_average_oracle(points, weights)
            if abs(self.geometric_median_objective(median, points, alphas) - self.geometric_median_objective(new_median, points, alphas)) < ftol:
                break
            median = new_median
        return median

    def aggregation(self, weights_results, type_aggregation):
        if type_aggregation == "mean":
            parameters_aggregated = ndarrays_to_parameters(
                flwr.server.strategy.aggregate.aggregate(weights_results)
            )
        elif type_aggregation == "median":
            # get input for RFA
            update_vectors = [np.concatenate([w.flatten() for w in fl.common.parameters_to_ndarrays(fit_res.parameters)]) for _, fit_res in weights_results]
            num_examples = [fit_res.num_examples for _, fit_res in weights_results]
    
            # aggregation RFA
            median_vector = self.geometric_median_update(update_vectors, num_examples)
    
            parameters = get_parameters(self.net)
            shapes = [w.shape for w in parameters]
            sizes = [int(np.prod(s)) for s in shapes]
            
            slices = np.split(median_vector, np.cumsum(sizes)[:-1])
            median_weights = [s.reshape(shape) for s, shape in zip(slices, shapes)]
            
            parameters_aggregated = ndarrays_to_parameters(median_weights)

        return parameters_aggregated

    def aggregate_fit(self, server_round: int, results: List[Tuple[ClientProxy, FitRes]], failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]]):
        
        # aggregated = super().aggregate_fit(server_round, results, failures)
        aggregated = self.aggregation(weights_results=results, type_aggregation="median") # params

        if aggregated is not None:
        # if aggregated and aggregated[0] is not None:
        #     aggregated_ndarrays: list[np.ndarray] = fl.common.parameters_to_ndarrays(
        #         aggregated[0]
        #     )

            # params_dict = zip(self.net.state_dict().keys(), aggregated_ndarrays)
            aggregated_ndarrays = fl.common.parameters_to_ndarrays(aggregated)
            params_dict = zip(self.net.state_dict().keys(), aggregated_ndarrays)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            self.net.load_state_dict(state_dict, strict=True)

            # Save the model to disk
            torch.save(self.net.state_dict(), f"model_round_{server_round}.pth")

        aggregated_metrics = {}
        if results:
            metrics = [(res.num_examples, res.metrics) for _, res in results]
            # update global prototypes
            for _, res in results:
                label_counts = pickle.loads(res.metrics["label_counts"])
                local_prototypes = pickle.loads(res.metrics["local_prototypes"])
                for label in label_counts:
                    proto = local_prototypes[label].to("cpu")
                    count = label_counts[label]
                    self.global_prototypes[label] += (count * proto) / res.num_examples
                    
            aggregated_metrics = self.weighted_average(metrics)
            self.fit_metrics_history.append(aggregated_metrics)

            if wandb.run is not None:
                wandb.log({
                    "train_server_round": server_round, 
                    "train_accuracy": aggregated_metrics.get("train_accuracy", 0.0), 
                    "train_loss" : aggregated_metrics.get("train_loss", 0.0)
                })

            print(f"Round {server_round} training metrics: {aggregated_metrics}")

        return aggregated, aggregated_metrics

    def aggregate_evaluate( self, server_round: int, results: List[Tuple[ClientProxy, EvaluateRes]],  failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]]):
        
        aggregated = super().aggregate_evaluate(server_round, results, failures)

        if results:
            metrics = [(res.num_examples, res.metrics) for _, res in results]
            aggregated_metrics = self.weighted_average(metrics)
            self.eval_metrics_history.append(aggregated_metrics)
            
            if wandb.run is not None:
                wandb.log({
                    "eval_server_round" : server_round,
                    "test_accuracy": aggregated_metrics.get("test_accuracy"), 
                    "test_loss" : aggregated_metrics.get("test_loss"), 
                })
                
            print(f"Round {server_round} evaluation metrics: {aggregated_metrics}")

        return aggregated

    def get_dropout_history(self) -> Dict[int, List[int]]:
        return self.dropped_clients_history_training, self.dropped_clients_history_evaluation

    def get_metrics_history(self) -> Tuple[List[Dict[str, float]], List[Dict[str, float]]]: 
        return self.fit_metrics_history, self.eval_metrics_history

In [235]:
from flwr.client import ClientApp
from flwr.server import ServerApp
from flwr.simulation import run_simulation
import os
from flwr.common import Context

In [236]:
def run_dropout_experiment(
    client_fn_creator,
    pl_model : Union[pl.LightningModule, torch.nn.Module], 
    num_clients: int,
    num_rounds: int = 5,
    dropout_rate_training: float = 0.3,
    dropout_rate_eval: float = 0.3,
    dropout_pattern_train: str = "random",
    dropout_pattern_eval: str = "random",
    fixed_clients: Optional[List[int]] = None,
    experiment_name: str = "dropout_experiment",
    save_dir: str = "model_weights",
    num_gpus : int = 0, 
    resource_config : Optional[Dict[str, float]] = None,

):
    
    # Configure client app
    print(f"\nStarting experiment: {experiment_name}")
    print(f"Dropout rate training: {dropout_rate_training}, Pattern: {dropout_pattern_train}")
    print(f"Dropout rate evaluation: {dropout_rate_eval}, Pattern: {dropout_pattern_eval   }")
    print(f"Number of GPUs: {num_gpus}")
    print(f"Number of clients: {num_clients}")
    print(f"Number of rounds: {num_rounds}")
    print(f"Fixed clients: {fixed_clients or []}")
    # Create strategy with dropout
    strategy = DropoutFedAvg(
        net=pl_model.model if isinstance(pl_model, pl.LightningModule) else pl_model,
        dropout_rate_training=dropout_rate_training,
        dropout_rate_eval=dropout_rate_eval,
        dropout_pattern_train=dropout_pattern_train,
        dropout_pattern_eval=dropout_pattern_eval,
        fixed_clients=fixed_clients or [],
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=1,
        min_evaluate_clients=1,
        min_available_clients=1,

    )

    # Configure server with strategy
    def server_fn(server_context: Context):
        from flwr.server import ServerAppComponents, ServerConfig
        config = ServerConfig(num_rounds=num_rounds)
        return ServerAppComponents(strategy=strategy, config=config)


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    epochs = resource_config.get("epochs", 1) if resource_config else 1
    client_datasets = resource_config.get("client_datasets", {}) if resource_config else {}

    
    batch_size = resource_config.get("batch_size", 32) if resource_config else 32
    learning_rate = resource_config.get("learning_rate", 0.001) if resource_config else 0.001
    num_workers = resource_config.get("num_workers", 1) if resource_config else 1
    client_fn = client_fn_creator(device=device, epochs=epochs, client_datasets=client_datasets
                                , batch_size=batch_size, pl_model=pl_model, num_workers=num_workers)
    
    # Create client and server apps
    client_app = ClientApp(client_fn=client_fn)
    server_app = ServerApp(server_fn=server_fn)

    # Configure backend
    backend_config = {
        "client_resources": {
            "num_cpus": 1,
            "num_gpus": num_gpus,
        }
    }
    history = strategy.get_dropout_history()
    # Run simulation
    try:
        run_simulation(
            client_app=client_app,
            server_app=server_app,
            num_supernodes=num_clients,
            backend_config=backend_config,
        )

        # Get metrics directly from strategy
        fit_metrics, eval_metrics = strategy.get_metrics_history()

        # Format metrics for plotting
        rounds = list(range(1, len(eval_metrics) + 1))

        train_accuracy_values = [metrics.get("train_accuracy", 0.0) for metrics in fit_metrics]
        train_loss_values = [metrics.get("train_loss", 0.0) for metrics in fit_metrics]
        

        test_accuracy_values = [metrics.get("test_accuracy", 0.0) for metrics in eval_metrics]
        test_loss_values = [metrics.get("test_loss", 0.0) for metrics in eval_metrics]


        # cleanup_wandb_loggers()
        results = {
            "rounds": rounds,
            "train_accuracy": train_accuracy_values,
            "train_loss": train_loss_values,
            "test_accuracy": test_accuracy_values,
            "test_loss": test_loss_values,
        }

        return results, history
    
    except Exception as e:
        print(f"Error in dropout experiment: {e}")
        import traceback
        traceback.print_exc()
        return {"error": str(e)}

# Main

In [237]:
import hydra 
from omegaconf import DictConfig, OmegaConf
import logging 
import wandb
import warnings

## Config 

In [238]:
config = {
    "base_path": "/kaggle",
    "device": "cuda",  # from train.device
    "run_name": "dropout_30pct_fixed_train_fixed_eval_0.3_fixed_0.3",  # resolved using experiment values
    "seed": 42,
    "num_clients": 4,
    "num_rounds": 3,
    "gpus": 1,
    "wandb_log": True,
    "train": {
        "batch_size": 4,
        "learning_rate": 0.001,
        "epochs": 5,
        "device": "cuda",
        "num_workers": 2,
        "weight_decay": 0.0001,
        "scheduler": {
            "use": False,
            "type": "cosine",
            "warmup_epochs": 5,
            "min_lr": 0.0001,
        }
    },

    "experiment": {
        "pattern_train": "fixed",
        "pattern_eval": "fixed",
        "dropout_rate_training": 0.3,
        "dropout_rate_eval": 0.3,
        "fixed_clients": [0, 1, 2],
        "name": "dropout_30pct_fixed_train_fixed_eval",
    },

    "data": {
        "root_path": "/kaggle/input/mri-dataset/datasetzip/not_skull_stripped",
        "label_path": "/kaggle/input/mri-label/label.csv",
        "val_ratio": 0.2,
        "overlap_ratio": 0.2,
        "distribution": "same",
    },
}


In [239]:
def run_experiment_with_lightning(cfg_dict: dict) -> None:
    cfg: DictConfig = OmegaConf.create(cfg_dict)  # convert to DictConfig to retain dot-access
    logger =  logger = logging.getLogger(__name__)
    logger.info(f"Running experiment with config: {cfg.experiment.name}")
    logger.info(f"Config: {OmegaConf.to_yaml(cfg)}")

    if cfg.wandb_log:
        wandb.init(
            project="sex-classification",
            name=f"{cfg.experiment.name}",
            config=OmegaConf.to_container(cfg, resolve=True),
            group="server"
        )

    device = cfg.device
    epochs = cfg.train.epochs

    logger.info("Loading model")
    net: nn.Module = DenseNet()
    pl_model: pl.LightningModule = DenseNetModule(net = net)

    logger.info("Loading dataset")
    is_3d = True if isinstance(pl_model, DenseNetModule) else False
    print(f"Is 3D: {is_3d}")

    full_dataset = MRIDataset(
        root_dir=cfg.data.root_path,
        label_path=cfg.data.label_path,
        is_3d=is_3d
    )

    logger.info(f"Dataset loaded successfully with len is {len(full_dataset)}")
    logger.info(f"Splitting dataset into {cfg.num_clients} clients")

    if cfg.data.distribution == "iid":
        client_datasets = iid_client_split(
            full_dataset,
            num_client=cfg.num_clients,
            val_ratio=cfg.data.val_ratio
        )
    elif cfg.data.distribution == "same":
        client_datasets = same_distribution_client_split(
            full_dataset,
            num_client=cfg.num_clients,
            val_ratio=cfg.data.val_ratio,
            overlap_ratio=cfg.data.overlap_ratio,
            root_dir=cfg.data.root_path,
            is_3d=is_3d
        )
    else:
        raise ValueError(f"Unknown distribution type: {cfg.data.distribution}")

    logger.info(f"Client datasets created successfully with {len(client_datasets)} clients")

    resources = {
        "client_datasets": client_datasets,
        "device": device,
        "epochs": epochs,
        "batch_size": cfg.train.batch_size,
        "learning_rate": cfg.train.learning_rate,
        "num_workers": cfg.train.num_workers
    }

    logger.info("Running experiments with PyTorch Lightning")

    results, history = run_dropout_experiment(
        client_fn_creator=create_lightning_client_fn,
        pl_model=pl_model,
        num_clients=cfg.num_clients,
        num_rounds=cfg.num_rounds,
        dropout_rate_training=cfg.experiment.dropout_rate_training,
        dropout_rate_eval=cfg.experiment.dropout_rate_eval,
        dropout_pattern_train=cfg.experiment.pattern_train,
        dropout_pattern_eval=cfg.experiment.pattern_eval,
        experiment_name=cfg.experiment.name,
        num_gpus=cfg.gpus,
        resource_config=resources
    )

    logger.info("Run successfully + wandb tracking")
    print(f"Result is {results}")

    if cfg.wandb_log:
        wandb.finish()

    logger.info("Experiments completed successfully")
    logger.info(f"Client Dropout History: {history}")

    return results, history

In [240]:
wandb.login(key=WANDB_APIKEY)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [241]:
run_experiment_with_lightning(config)

/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py:259: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}  # noqa


Is 3D: True


INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client



Starting experiment: dropout_30pct_fixed_train_fixed_eval
Dropout rate training: 0.3, Pattern: fixed
Dropout rate evaluation: 0.3, Pattern: fixed
Number of GPUs: 1
Number of clients: 4
Number of rounds: 3
Fixed clients: []


(pid=23331) 2025-05-21 18:26:41.255211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=23331) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=23331) E0000 00:00:1747852001.284175   23331 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=23331) E0000 00:00:1747852001.292346   23331 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 4)


...

(ClientAppActor pid=23331) ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
(ClientAppActor pid=23331) ┃        Test metric        ┃       DataLoader 0        ┃
(ClientAppActor pid=23331) ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
(ClientAppActor pid=23331) │         test/acc          │    0.5326460599899292     │
(ClientAppActor pid=23331) │         test/loss         │    0.7016363739967346     │
(ClientAppActor pid=23331) └───────────────────────────┴───────────────────────────┘


(ClientAppActor pid=23331) INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
(ClientAppActor pid=23331) INFO: GPU available: True (cuda), used: True
(ClientAppActor pid=23331) INFO: TPU available: False, using: 0 TPU cores
(ClientAppActor pid=23331) INFO: HPU available: False, using: 0 HPUs
(ClientAppActor pid=23331) INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 5673.61s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.7635967811424575
INFO :      		round 2: 1.1208644631380094
INFO :      		round 3: 0.6037026001307779
INFO :      	History (metrics, distributed, fit):
INFO :      	{'train_accuracy': [(1, 0.6459330149244463),
INFO :      	                    (2, 0.7141148220551642),
INFO :      	                    (3,

(ClientAppActor pid=23331) ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
(ClientAppActor pid=23331) ┃        Test metric        ┃       DataLoader 0        ┃
(ClientAppActor pid=23331) ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
(ClientAppActor pid=23331) │         test/acc          │    0.7272727489471436     │
(ClientAppActor pid=23331) │         test/loss         │    0.4592388868331909     │
(ClientAppActor pid=23331) └───────────────────────────┴───────────────────────────┘
Round 3 evaluation metrics: {'test_accuracy': 0.6167664834125314, 'test_loss': 0.6037026001307779}
Result is {'rounds': [1, 2, 3], 'train_accuracy': [0.6459330149244463, 0.7141148220551642, 0.781214197476705], 'train_loss': [0.625308164996964, 0.5748761412487076, 0.48577622572580975], 'test_accuracy': [0.4574850231944444, 0.5544910161438102, 0.6167664834125314], 'test_loss': [0.7635967811424575, 1.1208644631380094, 0.6037026001307779]}


eval_server_round,▁▅█
test_accuracy,▁▅█
test_loss,▃█▁
train_accuracy,▁▅█
train_loss,█▅▁
train_server_round,▁▅█
eval_server_round,3
test_accuracy,0.61677
test_loss,0.6037
train_accuracy,0.78121
train_loss,0.48578


({'rounds': [1, 2, 3],
  'train_accuracy': [0.6459330149244463,
   0.7141148220551642,
   0.781214197476705],
  'train_loss': [0.625308164996964, 0.5748761412487076, 0.48577622572580975],
  'test_accuracy': [0.4574850231944444,
   0.5544910161438102,
   0.6167664834125314],
  'test_loss': [0.7635967811424575, 1.1208644631380094, 0.6037026001307779]},
 ({1: [17353880090414495932],
   2: [17353880090414495932],
   3: [3199048786198958087]},
  {1: [17353880090414495932],
   2: [11251355939264284581],
   3: [17353880090414495932]}))